#### 사용자 정의 모델 클래스
- 부모클래스 : nn.Module  
- 필수오버라이딩  
    * _ _init_ _() : 모델 층 구성 즉, 설계  
    * forward() : 순방향 학습 진행 코드 구현
- 동적 모델
    * container 모듈 중 nn.ModuleList() 사용해서 동적으로 Layer 추가
        - forward 기능 미 제공
        - layer 인스턴스 요소 사이에 연관성 없음
        - layer 인스턴스 요소는 인덱싱으로 접근

In [27]:
## 모듈로딩 
import torch                                # 텐서 관련 모듈
import torch.nn as nn                       # 인공신경망 관련 모듈
import torch.nn.functional as F             # 인공신경망 관련 함수들 모듈 ( 손실함수, 활성화함수 등등)
import torch.optim as optim                 # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary               # 모델 구조 및 정보 관련모듈
from torchmetrics.regression import *       # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *   # 분류 성능 지표 관련 모듈

In [28]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

- 모델 설계 - 동적 모델 <hr>
    * 목표 : 은닉층의 개수가 동적인 모델
    * 조건
        * 입력층과 출력층 개수 동적 => 입력층의 입력값, 출력층의 출력값
        * 은닉층의 개수 동적 + 퍼셉트론 개수 고정 => 은닉층의 개수, 퍼셉트론 수 입력


In [29]:
# ---------------------------------------------------------
# 모델 이름 : DynamicModel
# 부모클래스 : nn.Module
# 매개 변수 : in_in, out_out, h_ins=[], n_outs=[]
# ---------------------------------------------------------

In [30]:
class DynamicModel(nn.Module):

    # 모델 구조 설계 함수 즉, 생성자 메서드
    def __init__(self, in_in, in_out, out_out, h_ins=[], h_outs=[]):
        super().__init__()

        self.in_layer=nn.Linear(in_in, h_ins[0] if len(h_ins) else in_out)
        self.hidden_layers = nn.ModuleList()

        for i in range(len(h_ins)):
            self.hidden_layers.append(nn.Linear(h_ins[i],h_outs[i] ))
            
        self.out_layer=nn.Linear(h_outs[-1] if len(h_outs) else in_out, out_out)
    
    # 학습 진행 콜백 메서드
    def forward(self, x):
        
        # 입력층
        y= self.in_layer(x)   # y= x1W1+x2W2+x3W3+b
        y=F.relu(y)            # 0<=y     ---------------> 죽은 relue ==> 해결하기위해 leakyReLu

        # 은닉층
        for hidden_layer in self.h_layers:
            y=hidden_layer(y)   # y= x1W1+x2W2+x3W3+x4W4......X5W5+b
            y=F.relu(y)

        # 출력층
        return self.out_layer(y)

In [45]:
class DynamicModel2(nn.Module):

    # 모델 구조 설계 함수 즉, 생성자 메서드
    def __init__(self, in_in, in_out, out_out, h_list=[]):
        super().__init__()

        self.in_layer=nn.Linear(in_in, h_list[0] if len(h_list) else in_out)
        self.hidden_layers = nn.ModuleList()

        for i in range(len(h_list)-1):
            self.hidden_layers.append(nn.Linear(h_list[i],h_list[i+1] ))
            
        self.out_layer=nn.Linear(h_list[-1] if len(h_list) else in_out, out_out)
    
    # 학습 진행 콜백 메서드
    def forward(self, x):
        
        # 입력층
        y= self.in_layer(x)   # y= x1W1+x2W2+x3W3+b
        y=F.relu(y)            # 0<=y     ---------------> 죽은 relue ==> 해결하기위해 leakyReLu

        # 은닉층
        for hidden_layer in self.hidden_layers:
            y=hidden_layer(y)   # y= x1W1+x2W2+x3W3+x4W4......X5W5+b
            y=F.relu(y)

        # 출력층
        return self.out_layer(y)

In [46]:
# 모델 인스턴스
### 모델 인스턴스 생성
h_ins, h_outs = [30, 50, 70], [50, 70, 30]
h_list = [30,50,70,90,110,50,60]
m1 = DynamicModel(3,5,2,h_ins, h_outs)
m2 = DynamicModel2(3,5,2, h_list)
m1
m2

DynamicModel2(
  (in_layer): Linear(in_features=3, out_features=30, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=30, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=70, bias=True)
    (2): Linear(in_features=70, out_features=90, bias=True)
    (3): Linear(in_features=90, out_features=110, bias=True)
    (4): Linear(in_features=110, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=60, bias=True)
  )
  (out_layer): Linear(in_features=60, out_features=2, bias=True)
)

In [47]:
print(m2)

DynamicModel2(
  (in_layer): Linear(in_features=3, out_features=30, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=30, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=70, bias=True)
    (2): Linear(in_features=70, out_features=90, bias=True)
    (3): Linear(in_features=90, out_features=110, bias=True)
    (4): Linear(in_features=110, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=60, bias=True)
  )
  (out_layer): Linear(in_features=60, out_features=2, bias=True)
)


In [48]:
for name, param in m1.named_parameters():
    print(name, param.shape)

in_layer.weight torch.Size([30, 3])
in_layer.bias torch.Size([30])
hidden_layers.0.weight torch.Size([50, 30])
hidden_layers.0.bias torch.Size([50])
hidden_layers.1.weight torch.Size([70, 50])
hidden_layers.1.bias torch.Size([70])
hidden_layers.2.weight torch.Size([30, 70])
hidden_layers.2.bias torch.Size([30])
out_layer.weight torch.Size([2, 30])
out_layer.bias torch.Size([2])


- 학습 진행

In [49]:
## 임시 데이터 생성
dataTS = torch.FloatTensor([[1,3,5],[2,4,6],[3,5,7], [4,6,8]]) # 2D (4,3) 4행3열
targetTS=torch.FloatTensor([[7,9],[8,10],[9,11],[10,12]])      # 2D (4,2) 4행2열



In [50]:
# 모델 학습
pre_y = m2(dataTS)
print(pre_y, pre_y.shape, sep='\n')

tensor([[-0.0424,  0.0235],
        [-0.0444,  0.0240],
        [-0.0466,  0.0250],
        [-0.0484,  0.0258]], grad_fn=<AddmmBackward0>)
torch.Size([4, 2])
